In [ ]:
from dataclasses import dataclass, field
from typing import List

@dataclass
class DNA_feature:
    name: str
    sequence: str
    start_position: int
    end_position: int
    chromosome: 'Chromosome'
    strand: str
    relative_pos: List[float]
    feature_type: str
    
    @property
    def length(self) -> int:
        return self.end_position - self.start_position + 1
    
    @property
    def relative_pos(self) -> float:
        return self.start_position / self.chromosome.size, self.end_position / self.chromosome.size

@dataclass
class Chromosome:
    name: str
    number: str  
    size: int  
    feature: List[DNA_feature] = field(default_factory=list)

    def add_feature(self, feature: DNA_feature):
        self.genes.feature(feature)

@dataclass
class Genome:
    species_name: str
    genome_name: str
    version: str
    chromosomes: List[Chromosome] = field(default_factory=list)
    
    @property
    def genome_size(self) -> int:
        return sum(chromosome.size for chromosome in self.chromosomes)
        
    @property
    def num_chromosomes(self) -> int:
        return len(self.chromosomes)
        
    def add_chromosome(self, chromosome: Chromosome):
        self.chromosomes.append(chromosome)


In [ ]:
# parse the gff3 file and populate the genome, chromosome, and DNA_feature objects
def parse_gff3_file(file_path):
    genome = Genome(species_name='', genome_name='', version='')
    current_chromosome = None
    
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            
            # Skip comment lines
            if line.startswith('#'):
                continue
            
            fields = line.split('\t')
            
            # Skip lines with incomplete fields
            if len(fields) < 9:
                continue
            
            seqid = fields[0]
            source = fields[1]
            feature_type = fields[2]
            start_position = int(fields[3])
            end_position = int(fields[4])
            strand = fields[6]
            attributes = fields[8]
            
            # Create a new chromosome if necessary
            if current_chromosome is None or current_chromosome.name != seqid:
                current_chromosome = Chromosome(name=seqid, number='', size=0)
                genome.add_chromosome(current_chromosome)
            
            # Create a new DNA feature
            dna_feature = DNA_feature(
                name='',
                sequence='',
                start_position=start_position,
                end_position=end_position,
                chromosome=current_chromosome,
                strand=strand,
                relative_pos=[],
                feature_type=feature_type
            )
            
            # Add the DNA feature to the chromosome
            current_chromosome.add_feature(dna_feature)
    
    return genome

# Usage example
file_path = './genome_example.gff3'
genome = parse_gff3_file(file_path)
